In [1]:
import os
os.chdir('../')

In [31]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [3]:
main_path = r'data/titanic'
train_p = r'train.csv'
test_p = r'test.csv'
gender_submission = r'gender_submission.csv'

In [4]:
train = pd.read_csv(os.path.join(main_path, train_p))
test = pd.read_csv(os.path.join(main_path, test_p))

In [5]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
features = ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
target = ['Survived']

In [7]:
Xtr, Ytr = train[features], train[target]
Xte = test[features]

In [8]:
whole = Xtr.append(Xte)

In [9]:
whole.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
le = preprocessing.LabelEncoder()
le.fit(np.unique(whole['Sex'].values))

whole['Sex'] = le.transform(whole['Sex'].values)

In [11]:
whole.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C


In [12]:
whole.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Name         1309 non-null   object 
 3   Sex          1309 non-null   int32  
 4   Age          1046 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Ticket       1309 non-null   object 
 8   Fare         1308 non-null   float64
 9   Cabin        295 non-null    object 
 10  Embarked     1307 non-null   object 
dtypes: float64(2), int32(1), int64(4), object(4)
memory usage: 117.6+ KB


In [13]:
names = whole.Name.values

In [14]:
whole['Title'] = list(map(lambda x: x.split(',')[1].lstrip(' ').split('.')[0], names))

In [15]:
title_mappings = {
    'Capt' : "Military",
    'Col' : "Military",
    'Don' : "Royalty",
    'Dona' : "Royalty",
    'Dr' : "Military",
    'Jonkheer' : "Royalty",
    'Lady' : "Royalty",
    'Major' : "Military",
    'Master' : "Master",
    'Miss' : "Miss",
    'Mlle' : "Miss",
    'Mme' : "Mrs",
    'Mr' : "Mr",
    'Mrs' : "Mrs",
    'Ms' : "Mrs",
    'Rev' : "Military",
    'Sir' : "Royalty",
    'the Countess' : "Royalty"
}

In [16]:
whole['Title'] = whole.Title.map(title_mappings)

In [17]:
whole.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S,Mr


In [18]:
whole.drop(['Name'], inplace=True, axis=1)

In [19]:
whole.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Sex          1309 non-null   int32  
 3   Age          1046 non-null   float64
 4   SibSp        1309 non-null   int64  
 5   Parch        1309 non-null   int64  
 6   Ticket       1309 non-null   object 
 7   Fare         1308 non-null   float64
 8   Cabin        295 non-null    object 
 9   Embarked     1307 non-null   object 
 10  Title        1309 non-null   object 
dtypes: float64(2), int32(1), int64(4), object(4)
memory usage: 117.6+ KB


In [20]:
whole.drop(['Cabin'], inplace=True, axis=1)
whole.drop(['Ticket'], inplace=True, axis=1)

In [21]:
whole.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,3,1,22.0,1,0,7.2500,S,Mr
1,2,1,0,38.0,1,0,71.2833,C,Mrs
2,3,3,0,26.0,0,0,7.9250,S,Miss
3,4,1,0,35.0,1,0,53.1000,S,Mrs
4,5,3,1,35.0,0,0,8.0500,S,Mr


In [22]:
whole.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Sex          1309 non-null   int32  
 3   Age          1046 non-null   float64
 4   SibSp        1309 non-null   int64  
 5   Parch        1309 non-null   int64  
 6   Fare         1308 non-null   float64
 7   Embarked     1307 non-null   object 
 8   Title        1309 non-null   object 
dtypes: float64(2), int32(1), int64(4), object(2)
memory usage: 97.2+ KB


In [23]:
whole['Age'].fillna(whole['Age'].median(), inplace=True)
whole['Embarked'].fillna(whole['Embarked'].values[100], inplace=True)
whole['Fare'].fillna(whole['Fare'].median(), inplace=True)

In [24]:
le = preprocessing.LabelEncoder()
le.fit(np.unique(whole['Embarked'].values))

whole['Embarked'] = le.transform(whole['Embarked'].values)

le = preprocessing.LabelEncoder()
le.fit(np.unique(whole['Title'].values))

whole['Title'] = le.transform(whole['Title'].values)

In [25]:
whole.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,3,1,22.0,1,0,7.2500,2,3
1,2,1,0,38.0,1,0,71.2833,0,4
2,3,3,0,26.0,0,0,7.9250,2,2
3,4,1,0,35.0,1,0,53.1000,2,4
4,5,3,1,35.0,0,0,8.0500,2,3


In [26]:
whole.shape

(1309, 9)

In [27]:
Xtr, Ytr = whole.iloc[:train.shape[0], :], train['Survived']
Xte = whole.iloc[train.shape[0]:, :]

In [28]:
ss = StandardScaler().fit(Xtr)

Xtr = ss.transform(Xtr)
Xte = ss.transform(Xte)

classifier = SVC()

#classifier = LogisticRegression(max_iter=10000)
#classifier = RandomForestClassifier()
#classifier = GradientBoostingClassifier()
classifier.fit(Xtr, Ytr.values)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [29]:
Xte.info()

AttributeError: 'numpy.ndarray' object has no attribute 'info'

In [30]:
result = classifier.predict(Xte)

In [45]:
ids = test['PassengerId'].values
output = np.vstack((ids, result))
output = pd.DataFrame(data=output.T, columns=['PassengerID', 'Survived'])
output_path = r'C:\Users\Wojtek\Documents\Projects\MLPlayground\output'
output.to_csv(os.path.join(output_path, "titanic.csv"), index=False)

In [57]:
p = r'C:\Users\Wojtek\Documents\Doktorat\Astral\data\Cont_AN_2_4\waves_morph.npy'

In [59]:
a = np.load(p)

In [60]:
a

array([[[ True,  True,  True, ..., False, False, False],
        [ True,  True, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ...,  True, False, False],
        [False, False, False, ..., False, False,  True]],

       [[False, False,  True, ..., False, False, False],
        [False,  True,  True, ..., False, False, False],
        [ True,  True, False, ...,  True, False, False],
        ...,
        [ True, False, False, ...,  True, False,  True],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False,  True]],

       [[False, False, False, ..., False, False, False],
        [False, False,  True, ..., False, False, False],
        [ True, False, False, ..., False, False, False],
        ...,
        [False, False, False, ...,  True, False,  True],
        [ True,  True, False, ..., False, Fal